In [37]:
import src.get_data as get_data

import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model, save_model
import datetime 
import seaborn as sns
from pylab import rcParams
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import time

import plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

import sqlalchemy

init_notebook_mode(connected=True)

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 14, 8

RANDOM_SEED = 42

WINDOW = 22

In [38]:

def load_data(X, seq_len, train_size=1):
    
    amount_of_features = X.shape[1] 
    X_mat = X.as_matrix() 
    
    data = []
    
    for index in range(len(X_mat)  - seq_len):
        data.append(X_mat[index: index + seq_len +1])
    
    data = np.array(data)
    train_split = int(round(train_size * data.shape[0]))
    train_data = data[:train_split, :]
    
    x_train = train_data[:, :-1]
    y_train = train_data[:, -1][:,-1]
    
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], amount_of_features))

    return x_train, y_train

"""
Собираем LSTM
"""

def build_model(input_shape):
    d = 0.2
    model = Sequential()
    
    model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(d))
        
    model.add(LSTM(128, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(d))
        
    model.add(Dense(32,kernel_initializer="normal",activation='relu'))        
    model.add(Dense(1,kernel_initializer="normal",activation='linear'))
    
    model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])
    
    return model

def predictNumDay(num, model, data, scaler):
    
    prediction = []
    lastbatch = np.array(data[-WINDOW:])
    for i in np.arange(num):    
        res = model.predict([lastbatch.reshape(1,22, 5)])
        prediction.append(scaler.inverse_transform(res))
    
    return np.array(prediction).reshape(num)


In [39]:

def nextDayPrediction(typeBlockchain, stock, N = 1):    
    
    df = get_data.get_data_frame(typeBlockchain, stock)

    x_scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()

    all_df = df.copy()

    x = all_df[['open', 'low', 'high', 'volume']].copy()
    y = all_df['close'].copy()

    x[['open', 'low', 'high', 'volume']] = x_scaler.fit_transform(x)

    y = y_scaler.fit_transform(y.values.reshape(-1, 1))
    x['close'] = y
    
    X_train, y_train = load_data(x, WINDOW)
    
    #print (X_train.shape, y_train.shape)
    
    model = build_model(input_shape=(WINDOW, 5))
    
    print('START FIT MODEL...')
    
    start = time.time()
    model.fit(X_train, y_train, batch_size=32, epochs=500,
              verbose=1)
    end = time.time()

    print ('Learning time: ', end-start)
    
    today = time.strftime("_%d_%m_%Y")
    
    pathModel = "../models/model_5f_" + typeBlockchain + today +".h5"
    save_model(model, pathModel)
    
    #model = load_model(pathModel)
    
    trainPredict = model.predict(X_train)
    trainPredict = y_scaler.inverse_transform(trainPredict)
    trainY = y_scaler.inverse_transform([y_train])

    trainScore = metrics.mean_squared_error(trainY[0], trainPredict[:,0]) ** .5
    print('Train Score: %.2f RMSE' % (trainScore))
    
    prices = df.close.values.astype('float32')
    prices = prices.reshape(len(prices), 1)
    
    trainPredictPlot = np.empty_like(prices)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[WINDOW:len(trainPredict)+WINDOW, :] = trainPredict

    """
    
    plt.plot(pd.DataFrame(prices, columns=["close"], index=df.index).close, label='Actual')
    plt.plot(pd.DataFrame(trainPredictPlot, columns=["close"], index=df.index).close, label='Training')
    plt.legend(loc='best')
    plt.show()
    
    """
    
    lastDate =str(df.date[df.last_valid_index()]).split('-')
    currentData = datetime.date(int(lastDate[0]),int(lastDate[1]),int(lastDate[2])) + datetime.timedelta(1)
    predictionDate = pd.date_range(currentData,periods=1)
    predictNday =  (predictNumDay(1, model, x, y_scaler))

    prediction = pd.DataFrame(predictNday, columns=["predictionPrice"], index = predictionDate.values)
    
    """Actual = pd.DataFrame(prices, columns=["close"], index=df.date).close
    Training = pd.DataFrame(trainPredictPlot, columns=["close"], index=df.date).close
    pred = pd.DataFrame(trainPredictPlot, columns=["close"], index=df.date).close

    ActualValues = go.Scatter( x = df.date, y = Actual, name = 'ActualValues')
    TrainingValues = go.Scatter( x = df.date, y = Training, name = 'TrainingValues')

    iplot([ActualValues,TrainingValues])
    
    our_Predict = go.Scatter( x = prediction.index, y = prediction.predictionPrice, name = 'NextDayValues')

    iplot([our_Predict, ActualValues])"""
    
    return prediction

In [40]:
USDT_BTC = nextDayPrediction('USDT_BTC', 'poloniex' ,N = 1)

START FIT MODEL...
Epoch 1/500
705/705 [==============================] - 1s - loss: 0.0398 - acc: 0.0014       
Epoch 2/500
705/705 [==============================] - 0s - loss: 0.0047 - acc: 0.0028     
Epoch 3/500
705/705 [==============================] - 0s - loss: 0.0016 - acc: 0.0028     
Epoch 4/500
705/705 [==============================] - 0s - loss: 0.0017 - acc: 0.0028     
Epoch 5/500
705/705 [==============================] - 0s - loss: 0.0014 - acc: 0.0028     
Epoch 6/500
705/705 [==============================] - 0s - loss: 0.0016 - acc: 0.0028      
Epoch 7/500
160/705 [=====>........................] - ETA: 0s - loss: 0.0030 - acc: 0.0000e+00

KeyboardInterrupt: 

In [8]:
USDT_BTC

,predictionPrice
2017-07-26,2581.344971


In [ ]:
USDT_LTC = nextDayPrediction('USDT_LTC', 'poloniex',N = 1)

2017-07-26 11:53:02,522 INFO sqlalchemy.engine.base.Engine select version()
2017-07-26 11:53:02,523 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 11:53:02,524 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-07-26 11:53:02,525 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 11:53:02,527 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-07-26 11:53:02,527 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 11:53:02,529 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-07-26 11:53:02,529 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 11:53:02,531 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-07-26 11:53:02,532 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 11:53:02,533 INFO sqlalchemy.engine.base.Engine SELECT date, time, open, close, low, high, volume, pair."name"FROM candlestick, pair WHERE candlestick.pair_id = pair.id AND pair.id IN (SELEC

In [ ]:
USDT_ETH = nextDayPrediction('USDT_ETH', 'poloniex',N = 1)

Learning time:  408.56155252456665
Train Score: 3.41 RMSE


In [ ]:
USDT_ETC = nextDayPrediction('USDT_ETC', 'poloniex',N = 1)

2017-07-26 12:06:48,766 INFO sqlalchemy.engine.base.Engine select version()
2017-07-26 12:06:48,767 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:06:48,768 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-07-26 12:06:48,769 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:06:48,770 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-07-26 12:06:48,771 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:06:48,773 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-07-26 12:06:48,773 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:06:48,775 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-07-26 12:06:48,775 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:06:48,776 INFO sqlalchemy.engine.base.Engine SELECT date, time, open, close, low, high, volume, pair."name"FROM candlestick, pair WHERE candlestick.pair_id = pair.id AND pair.id IN (SELEC

In [ ]:
USDT_XRP = nextDayPrediction('USDT_XRP', 'poloniex',N = 1)

2017-07-26 12:10:19,342 INFO sqlalchemy.engine.base.Engine select version()
2017-07-26 12:10:19,343 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:10:19,345 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-07-26 12:10:19,345 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:10:19,348 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-07-26 12:10:19,348 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:10:19,350 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-07-26 12:10:19,350 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:10:19,351 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-07-26 12:10:19,352 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:10:19,353 INFO sqlalchemy.engine.base.Engine SELECT date, time, open, close, low, high, volume, pair."name"FROM candlestick, pair WHERE candlestick.pair_id = pair.id AND pair.id IN (SELEC

In [86]:
print ('predictionValues on', datetime.date.today())

allCurrency = ['USDT_BTC', 'USDT_LTC', 'USDT_ETH', 'USDT_ETC', 'USDT_XRP']
predictedValues = [USDT_BTC, USDT_LTC, USDT_ETH, USDT_ETC, USDT_XRP]

for index in np.arange(len(allCurrency)):
    print(allCurrency[index], '\n' , predictedValues[index] )


predictionValues on 2017-07-26
USDT_BTC 
             predictionPrice
2017-07-26      2545.130371
USDT_LTC 
             predictionPrice
2017-07-26        42.890644
USDT_ETH 
             predictionPrice
2017-07-26       193.827957
USDT_ETC 
             predictionPrice
2017-07-26        14.239629
USDT_XRP 
             predictionPrice
2017-07-26         0.177823


In [63]:
def checkTrend(typeBlockchain, stock, predicted):
    
    df = get_data_frame(typeBlockchain, stock)
    lastValue = df.close[df.shape[0]-1]
    change = (predicted.values * 100 / lastValue) - 100
    if change > 0:
        mark = 'PositiveTrend'
    else: mark = 'NegativeTrend'
        
    print ('-'*10 + '\nSTOCK:%s, type:%s\n'%(stock, typeBlockchain))
    print ('Change between last (%f) and predicted values(%f) = %f perc.\n%s' \
           % (lastValue, predicted.values, change, mark))

In [84]:
stock = 'poloniex'

for index in np.arange(len(allCurrency)):
    checkTrend(allCurrency[index], stock , predictedValues[index] )

----------
STOCK:poloniex, type:USDT_BTC

Change between last (2565.999999) and predicted values(2545.130371) = -0.813316 perc.
NegativeTrend
----------
STOCK:poloniex, type:USDT_LTC

Change between last (42.110000) and predicted values(42.890644) = 1.853821 perc.
PositiveTrend
----------
STOCK:poloniex, type:USDT_ETH

Change between last (203.000000) and predicted values(193.827957) = -4.518250 perc.
NegativeTrend
----------
STOCK:poloniex, type:USDT_ETC

Change between last (14.620000) and predicted values(14.239629) = -2.601715 perc.
NegativeTrend
----------
STOCK:poloniex, type:USDT_XRP

Change between last (0.174540) and predicted values(0.177823) = 1.881226 perc.
PositiveTrend


In [8]:
!nvidia-smi

Fri Jul 28 12:44:21 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 375.66                 Driver Version: 375.66                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1070    Off  | 0000:01:00.0      On |                  N/A |
| 24%   54C    P2    45W / 151W |    604MiB /  8113MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [9]:
df = get_data.get_data_frame()

x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

all_df = df.copy()

x = all_df[['open', 'low', 'high', 'volume']].copy()
y = all_df['close'].copy()

x[['open', 'low', 'high', 'volume']] = x_scaler.fit_transform(x)

y = y_scaler.fit_transform(y.values.reshape(-1, 1))
x['close'] = y

X_train, y_train = load_data(x, WINDOW)

#print (X_train.shape, y_train.shape)

model = build_model(input_shape=(WINDOW, 5))



In [13]:
y_train.shape

(705,)

In [ ]:
print('START FIT MODEL...')

start = time.time()
model.fit(X_train, y_train, batch_size=32, epochs=500,
          verbose=1)

START FIT MODEL...
Epoch 1/500
705/705 [==============================] - 1s - loss: 0.0291 - acc: 0.0028     
Epoch 2/500
705/705 [==============================] - 0s - loss: 0.0033 - acc: 0.0028     
Epoch 3/500
705/705 [==============================] - 0s - loss: 0.0014 - acc: 0.0028     
Epoch 4/500
705/705 [==============================] - 0s - loss: 0.0019 - acc: 0.0028     
Epoch 5/500
705/705 [==============================] - 0s - loss: 0.0015 - acc: 0.0028     
Epoch 6/500
705/705 [==============================] - 0s - loss: 0.0014 - acc: 0.0028     
Epoch 7/500
705/705 [==============================] - 0s - loss: 0.0016 - acc: 0.0028     
Epoch 8/500
705/705 [==============================] - 0s - loss: 0.0015 - acc: 0.0028     
Epoch 9/500
705/705 [==============================] - 0s - loss: 0.0014 - acc: 0.0028      
Epoch 10/500
705/705 [==============================] - 0s - loss: 0.0012 - acc: 0.0028     
Epoch 11/500
705/705 [==============================] - 0s 

705/705 [==============================] - 0s - loss: 5.0356e-04 - acc: 0.0028     
Epoch 170/500
705/705 [==============================] - 0s - loss: 4.7507e-04 - acc: 0.0028     
Epoch 171/500
705/705 [==============================] - 0s - loss: 4.0478e-04 - acc: 0.0028     
Epoch 172/500
705/705 [==============================] - 0s - loss: 5.6579e-04 - acc: 0.0028     
Epoch 173/500
705/705 [==============================] - 0s - loss: 4.9723e-04 - acc: 0.0028      
Epoch 174/500
705/705 [==============================] - 0s - loss: 4.5842e-04 - acc: 0.0028     
Epoch 175/500
705/705 [==============================] - 0s - loss: 6.7579e-04 - acc: 0.0028     
Epoch 176/500
705/705 [==============================] - 0s - loss: 5.1759e-04 - acc: 0.0028     
Epoch 177/500
705/705 [==============================] - 0s - loss: 4.5358e-04 - acc: 0.0028     
Epoch 178/500
705/705 [==============================] - 0s - loss: 5.6387e-04 - acc: 0.0028       
Epoch 179/500
705/705 [========

705/705 [==============================] - 0s - loss: 5.4163e-04 - acc: 0.0028     
Epoch 336/500
705/705 [==============================] - 0s - loss: 0.0013 - acc: 0.0028     
Epoch 337/500
705/705 [==============================] - 0s - loss: 9.8329e-04 - acc: 0.0028 
Epoch 338/500
705/705 [==============================] - 0s - loss: 9.0683e-04 - acc: 0.0028     
Epoch 339/500
705/705 [==============================] - 0s - loss: 7.0558e-04 - acc: 0.0028     
Epoch 340/500
705/705 [==============================] - 0s - loss: 6.2762e-04 - acc: 0.0028     
Epoch 341/500
705/705 [==============================] - 0s - loss: 5.6465e-04 - acc: 0.0028     
Epoch 342/500
705/705 [==============================] - 0s - loss: 6.5895e-04 - acc: 0.0028     
Epoch 343/500
705/705 [==============================] - 0s - loss: 6.3254e-04 - acc: 0.0028     
Epoch 344/500
705/705 [==============================] - 0s - loss: 5.4999e-04 - acc: 0.0028     
Epoch 345/500
705/705 [===================

In [33]:
today = time.strftime("_%d_%m_%Y")
    

trainPredict = model.predict(X_train)
trainPredict = y_scaler.inverse_transform(trainPredict)
trainY = y_scaler.inverse_transform([y_train])

trainScore = metrics.mean_squared_error(trainY[0], trainPredict[:,0]) ** .5
print('Train Score: %.2f RMSE' % (trainScore))

prices = df.close.values.astype('float32')
prices = prices.reshape(len(prices), 1)

trainPredictPlot = np.empty_like(prices)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[WINDOW:len(trainPredict)+WINDOW, :] = trainPredict

"""

plt.plot(pd.DataFrame(prices, columns=["close"], index=df.index).close, label='Actual')
plt.plot(pd.DataFrame(trainPredictPlot, columns=["close"], index=df.index).close, label='Training')
plt.legend(loc='best')
plt.show()

"""
import datetime
lastDate =str(df.date[df.last_valid_index()]).split('-')
currentData = datetime.date(int(lastDate[0]),int(lastDate[1]),int(lastDate[2])) + datetime.timedelta(1)
predictionDate = pd.date_range(currentData,periods=1)
predictNday =  (predictNumDay(1, model, x, y_scaler))

prediction = pd.DataFrame(predictNday, columns=["predictionPrice"], index = predictionDate.values)

Train Score: 39.83 RMSE


In [35]:
df.tail()

,date,time,open,close,low,high,volume,name
722,2017-07-23,07:00:00,2847.000000,2751.430834,2655.000000,2880.151217,4.548646e+07,USDT_BTC
723,2017-07-24,07:00:00,2751.430834,2769.363636,2705.010000,2815.517448,3.298703e+07,USDT_BTC
724,2017-07-25,07:00:00,2769.363636,2565.999999,2417.330010,2800.000000,1.026457e+08,USDT_BTC
725,2017-07-26,07:00:00,2565.999999,2526.350647,2400.000000,2619.964935,5.210492e+07,USDT_BTC
726,2017-07-27,07:00:00,2526.350647,2669.999998,2506.440227,2695.651484,3.632087e+07,USDT_BTC
